In [1]:
import cProfile

from cfg import Cfg

from source import SourceCfg, Dimension, SpatialDistribution, AngularDistribution
from source import Source

from detector import DetectorCfg, Measurement

from sample import Sample, Layer, Material

from direct_problem import DirectProblemCfg
from direct_problem import DirectProblem


from support_functions import timeit, KW

import numpy as np

In [2]:
source_cfg = SourceCfg(dimension=Dimension.surface,
                       spatial_distribution=SpatialDistribution.gauss,
                       angular_distribution=AngularDistribution.collimated,
                       beam_center=np.array([0, 0, 0]),
                       beam_diameter=float(1))

detector_cfg = DetectorCfg(measurement=Measurement.CollimatedDiffuse)

direct_problem_cfg = DirectProblemCfg(N=10000,
                                      threads=1)

cfg = Cfg(
    source=source_cfg,
    detector=detector_cfg,
    direct_problem=direct_problem_cfg
    )

cfg.validate()

sample = Sample([
    Layer(material=Material.transparent, depth=1.,
          mu_a=0.0001, mu_s=0., g=0., n=1.45),
    Layer(material=Material.scattering, depth=1.,
          mu_a=1., mu_s=1., g=0.9, n=1.35),
    Layer(material=Material.transparent, depth=1.,
          mu_a=0.0001, mu_s=0., g=0.0, n=1.45)
          ])

detector = cfg.detector.get_detector()
source = Source(cfg.source, sample)

direct_problem = DirectProblem(cfg.direct_problem,
                               sample, source, detector)

result = timeit(direct_problem.solve)()
print(result)
print(np.sum(result[0]) + result[1])
print(KW)

(array([1249.43654936, 1830.90176432,   55.1288615 ,  341.81537123]), 6522.717453584516)
10000.000000000231
{'SOLVE': {'calls': 1, 'times': 3819}}


In [3]:
"""
Профилирование
ncalls – это количество совершенных вызовов;
tottime – это все время, потраченное в данной функции;
percall – ссылается на коэффициент tottime, деленный на ncalls;
cumtime – совокупное время, потраченное как в данной функции, так и наследуемых функциях. Это работает также и с рекурсивными функциями!
Второй столбец percall – это коэффициент cumtime деленный на примитивные вызовы;
filename:lineno(function) предоставляет соответствующие данные о каждой функции
"""
cProfile.run("direct_problem.solve()", sort = 'tottime')

         2053216 function calls (1973139 primitive calls) in 3.979 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    25730    0.652    0.000    1.625    0.000 direct_problem.py:98(move)
    45680    0.590    0.000    0.805    0.000 direct_problem.py:251(reflection)
     5054    0.344    0.000    0.445    0.000 ffi.py:149(__call__)
    15730    0.298    0.000    0.320    0.000 direct_problem.py:189(turn)
    10000    0.261    0.000    3.786    0.000 direct_problem.py:48(trace)
    20000    0.121    0.000    0.121    0.000 ufunclike.py:199(isneginf)
    43578    0.078    0.000    0.078    0.000 direct_problem.py:304(R_frenel)
46013/8214    0.063    0.000    0.089    0.000 ir.py:313(_rec_list_vars)
    75550    0.061    0.000    0.061    0.000 {method 'rand' of 'numpy.random.mtrand.RandomState' objects}
    25395    0.058    0.000    0.058    0.000 {method 'random' of 'numpy.random.mtrand.RandomState' objects}
   305668    0